In [1]:
from transformers import BertTokenizerFast

# Load the BERT tokenizer

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [4]:
# Function to process each element and write to the file
def process_and_write_to_file(element, file):
    # Split the text into lines
    lines = element.strip().split('\n')

    tokenized_sentences = []
    predictions = []
    labels = []

    spl = []
    for line in lines:
        spl.append(line.split())

    sen = [w[3] for w in spl if len(w) == 6]

    # Assuming 'tokenizer' is your tokenizer function
    tok = tokenizer(sen, is_split_into_words=True)
    tokens = tokenizer.convert_ids_to_tokens(tok['input_ids'])
    labels = [w[-1] for w in spl]
    prediction = [w[1] for w in spl]

    tokens = tokens[1:-1]

    # Align tokens and labels
    max_length = max(len(tokens), len(labels))
    tokens += [''] * (max_length - len(tokens))
    labels += [''] * (max_length - len(labels))

    # Write the aligned output to the file
    for token, label, pred in zip(tokens, labels, prediction):
        file.write(f"Prediction: {pred.ljust(10)} Token: {token.ljust(15)} Label: {label}\n")
    file.write("\n")  # Add a newline to separate each entry

# Read the content of the file
with open('err_analysis_formal_trans.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Split the content into a list based on empty lines
text_list = [part.strip() for part in file_content.strip().split('\n\n')]

# Open a text file for writing
with open("err_analysis_formal_trans.txt", "w", encoding='utf-8') as file:
    # Iterate through each element and process
    for i, element in enumerate(text_list, 1):
        process_and_write_to_file(element, file)

Testing the meteor score computation

In [ ]:
from googletrans import Translator

translator=Translator()

from nltk.translate import meteor_score

import nltk
nltk.download('wordnet')
import time

def meteor(sen, max_retries=3, timeout_seconds=10):
    for retry in range(max_retries):
        try:
            translated_text = translator.translate(sen, dest='it', timeout=timeout_seconds).text
            back_translated_text = translator.translate(translated_text, dest='en', timeout=timeout_seconds).text
            bsen = back_translated_text
            print(bsen)
            r = [sen.split()]
            c = bsen.split()
            meteor_score_value = meteor_score.meteor_score(r, c)

            return meteor_score_value, sen, bsen
        except Exception as e:
            print(f"An error occurred during translation (Retry {retry + 1}/{max_retries}): {e}")
            # Handle other exceptions if needed
            time.sleep(1)  # Add a delay before retrying

    print(f"Failed to translate after {max_retries} retries.")
    return 0, None, None

In [7]:
y,i,j = meteor('The early morning flight required them to hit the sack much earlier than usual')

The morning flight forced them to go to bed much earlier than usual


In [8]:
y

0.5919708677502443